In [1]:
from javascript import require, On, Once, AsyncTask, once, off, globalThis
import json
import llm_store

MINECRAFT_VERSION = '1.20.2'

collectDroppedItem = require('./control_primitives/collectDroppedItem.js')
craft = require('./control_primitives/craftItem.js')
explore = require('./control_primitives/exploreUntil.js')
move = require('./control_primitives/goToPosition.js')
kill = require('./control_primitives/killMob.js')
minePosition = require('./control_primitives/mineBlockPosition.js')
mineType = require('./control_primitives/mineBlockType.js')
place = require('./control_primitives/placeItem.js')
shoot = require('./control_primitives/shoot.js')
smelt = require('./control_primitives/smeltItem.js')
# getItemFromChest, depositItemIntoChest
chest = require('./control_primitives/useChest.js')

Vec3 = require('vec3').Vec3
mineflayer = require('mineflayer')
pathfinder = require('mineflayer-pathfinder')
pvp = require('mineflayer-pvp').plugin
mcData = require('minecraft-data')(MINECRAFT_VERSION)
armorManager = require("mineflayer-armor-manager")
autoeat = require('mineflayer-auto-eat').plugin
collectblock = require('mineflayer-collectblock').plugin
hawkeye = require('minecrafthawkeye')
toolPlugin = require('mineflayer-tool').plugin


RANGE_GOAL = 1

bot, movements = '', ''

# Load bot
def load_bot(username=None):
  # Create bot
  if not username:
    random_number = 176
    BOT_USERNAME = f'poo_bucket{random_number}'

  bot = mineflayer.createBot({ 
      'host': 'localhost',
      'port': 22222,
      'username': BOT_USERNAME, 
      'hideErrors': False 
  })

  once(bot, 'login')
  bot.chat('I spawned')

  # Load plugins
  bot.loadPlugin(collectblock)
  bot.loadPlugin(hawkeye)
  bot.loadPlugin(pathfinder.pathfinder)
  bot.loadPlugin(pvp)
  bot.loadPlugin(toolPlugin)

  globalThis.mcData = mcData
  globalThis.pathfinder = pathfinder
  movements = pathfinder.Movements(bot, mcData)
  bot.pathfinder.setMovements(movements)

  # Auto armor
  bot.loadPlugin(armorManager)
  bot.armorManager.equipAll()

  # Auto eat
  bot.loadPlugin(autoeat)
  bot.autoEat.options.priority = 'foodPoints'
  bot.autoEat.options.startAt = 14
  bot.autoEat.options.bannedFood.push('golden_apple')

  @On(bot, 'chat')
  def handleMsg(this, sender, message, *args):
      if sender and (sender != bot.username):
          bot.chat('Hi, you said ' + message)
          if 'come' in message:
              player = bot.players[sender]
              target = player.entity
              if not target:
                  bot.chat("I don't see you !")
                  return
              pos = target.position
              bot.pathfinder.setMovements(movements)
              bot.pathfinder.setGoal(pathfinder.goals.GoalNear(pos.x, pos.y, pos.z, RANGE_GOAL))
          if 'stop' in message:
              off(bot, 'chat', handleMsg)

  return bot, movements

In [2]:
bot, movements = load_bot()

[JSE] Mineflayer detected that you are using a deprecated event (physicTick)! Please use this event (physicsTick) instead.



In [3]:
CHESTS = [mcData.blocksByName[name]['id'] for name in mcData.blocksByName if 'chest' in name]
BEDS = [mcData.blocksByName[name]['id'] for name in mcData.blocksByName if 'bed' in name]

def getEnvironmentInfo(bot):
    def distanceTo(pos1, pos2):
        return ((pos1.x - pos2.x) ** 2 + (pos1.y - pos2.y) ** 2 + (pos1.z - pos2.z) ** 2) ** 0.5

    def nearbyEntities(bot, max_distance={'mobs': 32, 'items': 16, 'players': 128}):
        nearby_mobs, nearby_droppeditems, nearby_players = [], [], []
        for entity in bot.entities:
            if bot.entities[entity]:
                # Mob
                if bot.entities[entity].type not in ['other', 'player', 'projectile'] and distanceTo(bot.entities[entity].position, bot.entity.position) < max_distance['mobs']:
                    if bot.canSeeBlock(bot.blockAt(bot.entities[entity].position)):
                        nearby_mobs.append(f'{bot.entities[entity].type} {bot.entities[entity].name} is {int(distanceTo(bot.entities[entity].position, bot.entity.position))} blocks away.')
                # Item
                elif bot.entities[entity].displayName == 'Item' and distanceTo(bot.entities[entity].position, bot.entity.position) < max_distance['items']:
                    if bot.canSeeBlock(bot.blockAt(bot.entities[entity].position)):

                        nearby_droppeditems.append(f'DroppedItem {mcData.items[bot.entities[entity].metadata[8].itemId].displayName} is {int(distanceTo(bot.entities[entity].position, bot.entity.position))} blocks away.')
                # Player
                elif bot.entities[entity].type == 'player' and bot.entities[entity].username != bot.username and distanceTo(bot.entities[entity].position, bot.entity.position) < max_distance['players']:
                    if bot.canSeeBlock(bot.blockAt(bot.entities[entity].position)):

                        nearby_players.append(f'Player {bot.entities[entity].username} is {int(distanceTo(bot.entities[entity].position, bot.entity.position))} blocks away.')
        
        return nearby_mobs, nearby_droppeditems, nearby_players


    def nearbyBeds(bot, beds, max_distance=24, count=1):
        beds = bot.findBlocks({'matching': beds, 'maxDistance': max_distance, 'count': count})
        return [f'Bed at Vec3{bed.x, bed.y, bed.z} is {int(distanceTo(bed, bot.entity.position))} blocks away.' for bed in beds]

    def nearbyChests(bot, chests, max_distance=24, count=1):
        chests = bot.findBlocks({'matching': chests, 'maxDistance': max_distance, 'count': count})
        return [f'Chest at Vec3{chest.x, chest.y, chest.z} is {int(distanceTo(chest, bot.entity.position))} blocks away.' for chest in chests]
    
    nearby_mobs, nearby_droppeditems, nearby_players = nearbyEntities(bot, max_distance={'mobs': 64, 'items': 32, 'players': 128})

    nearby_beds = nearbyBeds(bot, BEDS, 24)
    nearby_chests = nearbyChests(bot, CHESTS, 24)

    return {
        'mobs': nearby_mobs,
        'droppedItems': nearby_droppeditems,
        'players': nearby_players,
        'beds': nearby_beds,
        'chests': nearby_chests
    }

def getWorldInfo(bot):
    # Time
    time = bot.time.timeOfDay
    if 0 <= time < 9000:
        time_description = 'in the Morning'
    elif 9000 <= time < 12000:
        time_description = 'at Dusk'
    else:
        time_description = 'at Night'
    
    # Weather
    weather = 'It is clear.' 
    if bot.rainState:
        weather = 'It is raining and thundering!' if bot.thunderState else 'It is raining!'
    
    return {
        'time': f'It is currently {time} {time_description}',
        'weather': weather,
        'biome': mcData.biomes[bot.blockAt(bot.entity.position).biome.id].displayName
    }

def getPlayerState(bot):
    # Inventory
    inventory = []
    for item in bot.inventory.items():
        inventory.append(f'{item.displayName} x{item.count}')
    if len(inventory) == 0: 
        inventory = 'Empty'

    # Held Item
    heldItem = 'Nothing'
    if bot.heldItem:
        heldItem = bot.heldItem.displayName

    return {
        'position': f'Vec3{bot.entity.position.x, bot.entity.position.y, bot.entity.position.z}',
        'health': bot.health,
        'hunger': bot.food,
        'heldItem': heldItem,
        'inventory': inventory
    }

def getPromptInfo(bot):
    world_info = getWorldInfo(bot)
    environment_info = getEnvironmentInfo(bot)
    player_state = getPlayerState(bot)

    return {
        'world_info': world_info,
        'environment_info': environment_info,
        'player_state': player_state
    }

In [4]:
with open('descriptions.json', 'r') as file:
    descriptions = json.load(file)
    
llms = llm_store.get_llms()

In [7]:
current_environment = getPromptInfo(bot)
print(current_environment)
plan = llms['planning_prompt'].invoke({
    'bot_actions': descriptions,
    'current_environment': current_environment,
    'previous_actions': ['Acquire food by hunting and cooking animals nearby... successfully acquired food.']
}).content
print(plan)

{'world_info': {'time': 'It is currently 7005 in the Morning', 'weather': 'It is clear.', 'biome': 'Plains'}, 'environment_info': {'mobs': ['animal pig is 54 blocks away.'], 'droppedItems': [], 'players': ['Player sillyguy69 is 2 blocks away.'], 'beds': [], 'chests': ['Chest at Vec3(1, 70, -50) is 7 blocks away.']}, 'player_state': {'position': 'Vec3(8.5, 68, -49.5)', 'health': 20, 'hunger': 20, 'heldItem': 'Netherite Sword', 'inventory': ['Netherite Sword x1', 'Diamond Pickaxe x1', 'Diamond Axe x1', 'Torch x64', 'Oak Log x64', 'Steak x64']}}
CURRENT GOAL: Acquire additional resources by mining coal and iron ore.

Given the inventory contents, which include a Diamond Pickaxe, the bot is well-equipped to mine for valuable resources like coal and iron ore. These resources are essential for crafting, smelting, and further tool creation.

### Step by Step Plan to Accomplish the Goal ###

1. **Locate Coal and Iron Ore**: Use the `explore.exploreUntil` function to move around the environment

In [10]:
mineType.mineBlockType(bot, 'coal_ore', 10, timeout=500000)

10 blocks found



In [11]:
getPromptInfo(bot)

{'world_info': {'time': 'It is currently 10025 at Dusk',
  'weather': 'It is clear.',
  'biome': 'Plains'},
 'environment_info': {'mobs': [],
  'droppedItems': [],
  'players': ['Player sillyguy69 is 2 blocks away.'],
  'beds': [],
  'chests': ['Chest at Vec3(1, 70, -50) is 8 blocks away.']},
 'player_state': {'position': 'Vec3(4.389287452210117, 63, -47.497276031672385)',
  'health': 20,
  'hunger': 20,
  'heldItem': 'Diamond Pickaxe',
  'inventory': ['Netherite Sword x1',
   'Diamond Pickaxe x1',
   'Diamond Axe x1',
   'Torch x64',
   'Oak Log x64',
   'Steak x64',
   'Dirt x1',
   'Cobblestone x8',
   'Coal x6']}}